## Artificial Neural Network (ANN)

10/22/2020

---

In [3]:
# Import libraries

import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [20]:
# Import dataset

os.chdir('//Users/mareksturek/Documents/GitHub/deep-learning/data')
dataset = pd.read_csv('rawchurnmodelling.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [8]:
# Create X, y

X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [9]:
# Encoding categorical data

#Label Enconding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

#Label Enconding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [10]:
# Splitting the dataset into the training & test set 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
# Feature scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# Building the ANN

# Initializing the ANN
ann = tf.keras.models.Sequential()

In [13]:
# Adding the input layer and the first hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [14]:
# Adding the second hidden layer

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [15]:
# Adding the output layer

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [16]:
# Training the ANN

# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
# Training the ANN on the training set

ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 883us/step - loss: 0.5002 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 951us/step - loss: 0.4542 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 944us/step - loss: 0.4415 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 971us/step - loss: 0.4355 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 888us/step - loss: 0.4321 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 951us/step - loss: 0.4292 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 893us/step - loss: 0.4265 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 930us/step - loss: 0.4241 - accuracy: 0.7979
Epoch 9/100
250/250 [==============================] - 0s 901us/step - loss: 0.4216 - accuracy: 0.8052
Epoch 10/100
250/250 [==============================] - 0s 926us/step - l

### Predicting the result of a single observation

An example choice is listed below:

- Geography: France
- Credit Score: 600
- Gender: Male
- Age: 40 years old
- Tenure: 3 years
- Balance: \$ 60000
- Number of Products: 2
- Does this customer have a credit card ? Yes
- Is this customer an Active Member: Yes
- Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [21]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


In [22]:
# Predicting the test set results

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [23]:
# Confusion matrix

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1510   85]
 [ 193  212]]


0.861